In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table


In [2]:
# create engine to hawaii.sqlite
connect_string = "sqlite:///static/data/climateDB.db"

# reflect the tables
engine = create_engine(connect_string) 

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)


In [3]:
Base.classes.keys()

['CO2_emission', 'country_demo', 'temp_change']

In [4]:
# Save references to each table
Emission = Base.classes.CO2_emission
Temp_change = Base.classes.temp_change
Country_demo = Base.classes.country_demo

In [5]:
# Create a session (link) from Python to the sqlite DB
session = Session(bind = engine)

#Filter the data for the year >= 1961
results_emission = session.query(Emission).filter(Emission.Year >= 1961)
#print(results) 

In [6]:
emission_df = pd.read_sql(results_emission.statement, session.connection())
results_temp = session.query(Temp_change)
temp_df = pd.read_sql(results_temp.statement, session.connection())



# filter Meteorological year
meteor_df = temp_df.loc[temp_df["Months"] == 'Meteorological year' ]
meteor_id_df = meteor_df.copy()

# Calculate avg temp per Meteorological year
meteor_id_df['avg_temp']= round(meteor_id_df.mean(axis =1),3)
meteor_id_df

,field1,Area,Months,Element,Unit,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp
16,32,Afghanistan,Meteorological year,Temperature change,°C,-0.080,-0.120,0.882,-0.727,-0.200,...,1.342,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893,0.958
33,66,Albania,Meteorological year,Temperature change,°C,0.631,0.339,0.084,-0.171,-0.392,...,1.123,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911,1.577
50,100,Algeria,Meteorological year,Temperature change,°C,0.186,0.101,0.045,0.212,-0.135,...,1.492,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224,2.366
67,168,Andorra,Meteorological year,Temperature change,°C,0.749,0.110,-0.755,0.305,-0.500,...,1.643,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871,3.480
84,202,Angola,Meteorological year,Temperature change,°C,0.053,-0.139,-0.176,-0.215,-0.181,...,0.708,0.401,0.910,0.726,1.260,1.534,0.808,1.264,1.694,3.773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3311,9518,Low Income Food Deficit Countries,Meteorological year,Temperature change,°C,-0.085,-0.186,0.072,-0.207,-0.224,...,0.872,0.618,0.875,0.820,1.061,1.292,1.276,1.094,1.244,159.063
3328,9552,Net Food Importing Developing Countries,Meteorological year,Temperature change,°C,-0.015,-0.096,0.116,-0.204,-0.155,...,0.878,0.692,0.977,0.971,1.269,1.374,1.292,1.222,1.412,159.672
3345,9586,Annex I countries,Meteorological year,Temperature change,°C,0.453,0.161,0.172,-0.366,-0.409,...,1.090,1.512,1.109,1.119,1.616,2.098,1.607,1.397,1.627,160.359
3362,9620,Non-Annex I countries,Meteorological year,Temperature change,°C,0.029,-0.059,0.149,-0.174,-0.092,...,0.801,0.768,0.955,1.003,1.276,1.366,1.316,1.234,1.361,160.793


In [7]:
# rename = ['Brunei ','DR Congo','Caribbean Netherlands']
# to_country = ['Brunei Darussalam', 'Democratic Republic of the Congo','Caribbean']
# for i in range(len(rename)):
#     print(to_country[i])
#     update_name = session.query(Country_demo).filter(Country_demo.name ==rename[i])
#     update_name.name = to_country[i]
# session.commit()
# update_name.name

In [8]:
results_Demo = session.query(Country_demo)
demo_df = pd.read_sql(results_Demo.statement, session.connection(),index_col='index')
session.close()

In [9]:
demo_sorted_df = demo_df.sort_values(by=["name"]).reset_index(drop=True)
demo_sorted_df = demo_sorted_df.rename({"name":"Area","population":"Population",
                                        "density":"Density",
                                        "land-size":"Land Size","image_url":"Images",
                                        "latitude":"Lat","longitude":"Lng"},axis='columns')
demo_sorted_df

,Area,Population,Density,Land Size,median_age,urban_pop,Images,Lat,Lng
0,Afghanistan,"38,928,346",60,"652,860",18,25 %,https://www.worldometers.info/img/flags/af-fla...,33.93911,67.709953
1,Albania,"2,877,797",105,"27,400",36,63 %,https://www.worldometers.info/img/flags/al-fla...,41.153332,20.168331
2,Algeria,"43,851,044",18,"2,381,740",29,73 %,https://www.worldometers.info/img/flags/ag-fla...,28.033886,1.659626
3,American Samoa,"55,191",276,200,N.A.,88 %,None,-14.270972,-170.132217
4,Andorra,"77,265",164,470,N.A.,88 %,https://www.worldometers.info/img/flags/an-fla...,42.546245,1.601554
...,...,...,...,...,...,...,...,...,...
230,Wallis & Futuna,"11,239",80,140,N.A.,0 %,None,None,None
231,Western Sahara,"597,339",2,"266,000",28,87 %,None,24.215527,-12.885834
232,Yemen,"29,825,964",56,"527,970",20,38 %,https://www.worldometers.info/img/flags/ym-fla...,15.552727,48.516388
233,Zambia,"18,383,955",25,"743,390",18,45 %,https://www.worldometers.info/img/flags/za-fla...,-13.133897,27.849332


In [10]:
# demo_countries = demo_sorted_df['name'].tolist()
# temp_countries = meteor_df['Area'].tolist()

# countries = []
# for country in temp_countries:
#     for demo in demo_countries:
#         if country== demo:
#             countries.append(country)
            
# len(countries)
# for country in temp_countries:
#     if country not in countries:
#         print(country)

In [11]:
# for country in demo_countries:
#     if country not in countries:
#         print(country)


In [12]:
#calculate overall avg_co2 emission per country
avg_co2 =  emission_df.groupby("Entity").agg({'AnnualCO2emissions':'mean'})
avg_co2 = round(avg_co2/1000000,3) ## converting GT to Mega ton for the tooltip
avg_co2.reset_index(inplace=True)
avg_co2

,Entity,AnnualCO2emissions
0,Afghanistan,3.036
1,Africa,726.391
2,Albania,4.503
3,Algeria,74.340
4,Andorra,0.246
...,...,...
229,Wallis and Futuna Islands,0.008
230,World,22816.567
231,Yemen,10.082
232,Zambia,3.289


In [13]:
avg_co2 = avg_co2.rename({'Entity':'Area', 'AnnualCO2emissions':'AnnualCO2emissions'}, axis='columns')

avg_co2

,Area,AnnualCO2emissions
0,Afghanistan,3.036
1,Africa,726.391
2,Albania,4.503
3,Algeria,74.340
4,Andorra,0.246
...,...,...
229,Wallis and Futuna Islands,0.008
230,World,22816.567
231,Yemen,10.082
232,Zambia,3.289


In [14]:
#Merge Temp_change by meteor year per country to Avg_Co2 Emission df
merged_co2_country =meteor_id_df.merge(avg_co2, how = 'left',  on="Area")

#find null
merged_co2_country.isna().sum()
#fill 0
merged_co2_country = merged_co2_country.fillna(0)
#find null again
merged_co2_country.isna().sum()

#merge population data to Temp and Co2 Emission df
popu_data = merged_co2_country.merge(demo_sorted_df, how = 'left', on ="Area" )

popu_data = popu_data[['Area', 'Population','Density', 'Land Size', 'Images', 'Lat','Lng']].fillna(0).set_index("Area",drop=True)
popu_data

,Population,Density,Land Size,Images,Lat,Lng
Area,,,,,,
Afghanistan,"38,928,346",60,"652,860",https://www.worldometers.info/img/flags/af-fla...,33.93911,67.709953
Albania,"2,877,797",105,"27,400",https://www.worldometers.info/img/flags/al-fla...,41.153332,20.168331
Algeria,"43,851,044",18,"2,381,740",https://www.worldometers.info/img/flags/ag-fla...,28.033886,1.659626
Andorra,"77,265",164,470,https://www.worldometers.info/img/flags/an-fla...,42.546245,1.601554
Angola,"32,866,272",26,"1,246,700",https://www.worldometers.info/img/flags/ao-fla...,-11.202692,17.873887
...,...,...,...,...,...,...
Low Income Food Deficit Countries,0,0,0,0,0,0
Net Food Importing Developing Countries,0,0,0,0,0,0
Annex I countries,0,0,0,0,0,0


In [15]:
merged_co2_country.set_index("Area",inplace=True)
merged_co2_country

,field1,Months,Element,Unit,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp,AnnualCO2emissions
Area,,,,,,,,,,,,,,,,,,,,,
Afghanistan,32,Meteorological year,Temperature change,°C,-0.080,-0.120,0.882,-0.727,-0.200,0.176,...,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893,0.958,3.036
Albania,66,Meteorological year,Temperature change,°C,0.631,0.339,0.084,-0.171,-0.392,0.551,...,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911,1.577,4.503
Algeria,100,Meteorological year,Temperature change,°C,0.186,0.101,0.045,0.212,-0.135,0.404,...,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224,2.366,74.340
Andorra,168,Meteorological year,Temperature change,°C,0.749,0.110,-0.755,0.305,-0.500,0.403,...,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871,3.480,0.246
Angola,202,Meteorological year,Temperature change,°C,0.053,-0.139,-0.176,-0.215,-0.181,0.193,...,0.401,0.910,0.726,1.260,1.534,0.808,1.264,1.694,3.773,11.971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Low Income Food Deficit Countries,9518,Meteorological year,Temperature change,°C,-0.085,-0.186,0.072,-0.207,-0.224,0.080,...,0.618,0.875,0.820,1.061,1.292,1.276,1.094,1.244,159.063,0.000
Net Food Importing Developing Countries,9552,Meteorological year,Temperature change,°C,-0.015,-0.096,0.116,-0.204,-0.155,0.129,...,0.692,0.977,0.971,1.269,1.374,1.292,1.222,1.412,159.672,0.000
Annex I countries,9586,Meteorological year,Temperature change,°C,0.453,0.161,0.172,-0.366,-0.409,0.270,...,1.512,1.109,1.119,1.616,2.098,1.607,1.397,1.627,160.359,0.000


In [16]:
New_Countires = merged_co2_country.index
New_Countires


Index(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Anguilla',
       'Antarctica', 'Argentina', 'Australia', 'Austria',
       ...
       'Polynesia', 'European Union', 'Least Developed Countries',
       'Land Locked Developing Countries', 'Small Island Developing States',
       'Low Income Food Deficit Countries',
       'Net Food Importing Developing Countries', 'Annex I countries',
       'Non-Annex I countries', 'OECD'],
      dtype='object', name='Area', length=201)

In [17]:


#Create a dictionary holding above values
 #meta ={
#    'country' : country_name,
#    'demo_info' : [web scraped data],
#    'tool_tip' : [{'c_name': country_name,
#                'avg_temp':avg_temp,
#                'avg_co2' : new_Avg_c02,
#                'population':population from web scraping
#                }]
#   } 

#New Code---------------------------------------------------

meta = []

for country in New_Countires:
    temp_co2_obj = {
                    "Country":country,
                    "Avg Temp Change":merged_co2_country.loc[country,"avg_temp"],
                    "Avg Co2 Change":merged_co2_country.loc[country,"AnnualCO2emissions"],
                    "Population":popu_data.loc[country,"Population"],
                    "Density":popu_data.loc[country,"Density"],
                    "Land Size":popu_data.loc[country,"Land Size"],
                    "Images":popu_data.loc[country,"Images"],
                    "Lat":popu_data.loc[country,"Lat"],
                    "Lng":popu_data.loc[country,"Lng"],
                    }
    meta.append(temp_co2_obj)
meta

[{'Country': 'Afghanistan',
  'Avg Temp Change': 0.958,
  'Avg Co2 Change': 3.036,
  'Population': '38,928,346',
  'Density': '60',
  'Land Size': '652,860',
  'Images': 'https://www.worldometers.info/img/flags/af-flag.gif',
  'Lat': '33.93911',
  'Lng': '67.709953'},
 {'Country': 'Albania',
  'Avg Temp Change': 1.577,
  'Avg Co2 Change': 4.503,
  'Population': '2,877,797',
  'Density': '105',
  'Land Size': '27,400',
  'Images': 'https://www.worldometers.info/img/flags/al-flag.gif',
  'Lat': '41.153332',
  'Lng': '20.168331'},
 {'Country': 'Algeria',
  'Avg Temp Change': 2.366,
  'Avg Co2 Change': 74.34,
  'Population': '43,851,044',
  'Density': '18',
  'Land Size': '2,381,740',
  'Images': 'https://www.worldometers.info/img/flags/ag-flag.gif',
  'Lat': '28.033886',
  'Lng': '1.659626'},
 {'Country': 'Andorra',
  'Avg Temp Change': 3.48,
  'Avg Co2 Change': 0.246,
  'Population': '77,265',
  'Density': '164',
  'Land Size': '470',
  'Images': 'https://www.worldometers.info/img/flags/